### <center>Signal Processing</center>

**Note**: _This file is only for testing purposes. It is not user-friendly and requires user to adjust it for themselves._


In [ ]:
%pip install -r "D:\\natHacks\\NeuroKnights\\requirements.txt"

#### Imports


In [1]:
import time
import serial.tools.list_ports

from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds

#### Listing available ports


In [2]:
ports = serial.tools.list_ports.comports()
print([port.device for port in ports])

['COM5', 'COM6', 'COM4', 'COM3']


In [3]:
BoardShim.enable_dev_board_logger()

params = BrainFlowInputParams()

params.serial_port = "COM6"

board = BoardShim(BoardIds.NEUROPAWN_KNIGHT_BOARD, params)

In [ ]:
print(board.get_sampling_rate(BoardIds.NEUROPAWN_KNIGHT_BOARD))
# print(board.get_eeg_channels(-1))

In [4]:
board.prepare_session()
print(board.is_prepared())

True


In [5]:
board.start_stream(450000)

In [ ]:
print(
    f"Indecies for EEG channels {BoardShim.get_eeg_channels(BoardIds.NEUROPAWN_KNIGHT_BOARD)}"
)
print(
    f"Indecies for EXG channels {BoardShim.get_exg_channels(BoardIds.NEUROPAWN_KNIGHT_BOARD)}"
)

In [6]:
channels = list(range(1, 4))  # TODO: change to using get_exg_channels
gain_value = 12

In [7]:
for channel in channels:
    board.config_board(f"chon_{channel}_{gain_value}")
    print(f"Enabled channel {channel} with gain {gain_value}.")
    time.sleep(0.5)

Enabled channel 1 with gain 12.
Enabled channel 2 with gain 12.
Enabled channel 3 with gain 12.


In [24]:
assert board
data = board.get_current_board_data(1000)[1:4]
data

array([[ 698.80493164, -187.27781677,  569.34362793, ...,  802.15942383,
        -342.96517944,  747.52178955],
       [ 688.67218018, -173.60848999,  555.51538086, ...,  799.89440918,
        -334.77944946,  740.84606934],
       [ 701.14941406, -194.98667908,  572.24438477, ...,  803.15283203,
        -345.86593628,  749.82653809]])

In [ ]:
data.shape

#### Attempting to implement analysis

**Recreating the method described in the study**: [Classification of SSVEP signals using the combined FoCCA-KNN method and comparison with other machine learning methods](https://www.sciencedirect.com/science/article/abs/pii/S1746809423003907)

**Electrode placement**: "OZ", "O1", "O2"


In [ ]:
from brainflow.data_filter import DataFilter, FilterTypes

##### Band-stop and band-pass filters


In [ ]:
sampling_rate = BoardShim.get_sampling_rate(board_id=BoardIds.NEUROPAWN_KNIGHT_BOARD)
DataFilter.perform_bandpass(
    data=data,
    sampling_rate=sampling_rate,
    start_freq=6.0,
    stop_freq=40.0,
    order=2,
    filter_type=FilterTypes.BUTTERWORTH_ZERO_PHASE.value,
    ripple=0,
)
# DataFilter.perform_bandstop(  # is this neccesary? we already have bandpass...
#     data=data,
#     sampling_rate=sampling_rate,
#     start_freq=49.0,
#     stop_freq=51.0,
#     order=2,
#     filter_type=FilterTypes.BUTTERWORTH_ZERO_PHASE.value,
#     ripple=0,
# )
# DataFilter.perform_bandstop(  # is this neccesary? we already have bandpass...
#     data=data,
#     sampling_rate=sampling_rate,
#     start_freq=59.0,
#     stop_freq=61.0,
#     order=2,
#     filter_type=FilterTypes.BUTTERWORTH_ZERO_PHASE.value,
#     ripple=0,
# )

##### Common Average Referencing (CAR) Filter


In [ ]:
data -= data.mean()

##### Selecting the appropriate harmonics, channels and coefficients


#### End session


In [ ]:
for channel in channels:
    board.config_board(f"choff_{channel}")
    print(f"Disabled channel {channel} with gain {gain_value}.")
    time.sleep(0.5)

In [ ]:
board.stop_stream()
board.release_session()